In [2]:
!pip install selenium


In [3]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

In [19]:
def get_jobs(keyword, num_jobs, verbose):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    #driver = webdriver.Chrome(executable_path="/Users/omersakarya/Documents/GitHub/scraping-glassdoor-selenium/chromedriver", options=options)
    driver = webdriver.Chrome(executable_path="C:/Users/Tavish Gandhi/Desktop/Ineuron/Salary_Project/chromedriver", options=options)
    
    driver.set_window_size(1120, 1000)
    url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="+keyword+"&sc.keyword="+keyword+"&locT=&locId=&jobType="
    #url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(15)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element_by_id("prefix__icon-close-1").click()
            driver.find_element_by_class_name("SVGInline-svg modal_closeIcon-svg").click() #clicking to the X.
        except NoSuchElementException:
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click()  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="gray salary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [18]:
df = get_jobs("data-scientist", 5, False)
df

Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Data Scientist,-1,If you are a data junkie who would like to wra...,3.0,"Blue Jeans Network, Inc.\n3.0",Bengaluru,-1,201 to 500 Employees,2009,Company - Private,Enterprise Software & Network Solutions,Information Technology,Unknown / Non-Applicable,-1
1,Data Scientist 1,-1,PayPals Next-Generation Platforms Consumer Ris...,3.7,"PayPal, Inc.\n3.7",Bengaluru,-1,10000+ Employees,1998,Company - Public,Internet,Information Technology,$10+ billion (USD),-1
2,Data Scientist,-1,Job Description:\n\n\nJob Title - Data Scienti...,3.5,Deutsche Bank AG\n3.5,Pune,-1,10000+ Employees,1870,Company - Public,Banks & Credit Unions,Finance,$10+ billion (USD),-1
3,Data Scientist,-1,Amazon strives to be Earth's most customer-cen...,4.3,"Amazon.com, Inc.\n4.3",Hyderabad,-1,10000+ Employees,1994,Company - Public,Internet,Information Technology,$10+ billion (USD),-1
4,Data Scientist,-1,Provide analytical insights into emerging prob...,3.7,"PayPal, Inc.\n3.7",Bengaluru,-1,10000+ Employees,1998,Company - Public,Internet,Information Technology,$10+ billion (USD),-1


In [20]:
df1 = get_jobs("data-scientist", 15, False)

Progress: 0/15
Progress: 1/15
Progress: 2/15
Progress: 3/15
Progress: 4/15
Progress: 5/15
Progress: 6/15
Progress: 7/15
Progress: 8/15
Progress: 9/15
Progress: 10/15
Progress: 11/15
Progress: 12/15
Progress: 13/15
Progress: 14/15
Progress: 15/15


In [21]:
df1


,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Data Scientist,-1,If you are a data junkie who would like to wra...,3.0,"Blue Jeans Network, Inc.\n3.0",Bengaluru,-1,201 to 500 Employees,2009,Company - Private,Enterprise Software & Network Solutions,Information Technology,Unknown / Non-Applicable,-1
1,Data Scientist 1,-1,PayPals Next-Generation Platforms Consumer Ris...,3.7,"PayPal, Inc.\n3.7",Bengaluru,-1,10000+ Employees,1998,Company - Public,Internet,Information Technology,$10+ billion (USD),-1
2,Data Scientist,-1,Amazon strives to be Earth's most customer-cen...,4.3,"Amazon.com, Inc.\n4.3",Hyderabad,-1,10000+ Employees,1994,Company - Public,Internet,Information Technology,$10+ billion (USD),-1
3,Data Scientist,-1,Job Description:\n\n\nJob Title - Data Scienti...,3.5,Deutsche Bank AG\n3.5,Pune,-1,10000+ Employees,1870,Company - Public,Banks & Credit Unions,Finance,$10+ billion (USD),-1
4,Data Scientist,-1,"When everything's connected, how we connect is...",3.4,TTEC\n3.4,Hyderabad,-1,10000+ Employees,1982,Company - Public,Staffing & Outsourcing,Business Services,$1 to $2 billion (USD),-1
5,Data Scientist,-1,Provide analytical insights into emerging prob...,3.7,"PayPal, Inc.\n3.7",Bengaluru,-1,10000+ Employees,1998,Company - Public,Internet,Information Technology,$10+ billion (USD),-1
6,Data Scientist,-1,Site Name: India - Karnataka - Bengaluru\nPost...,3.9,GlaxoSmithKline plc\n3.9,Bengaluru,-1,10000+ Employees,1830,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$10+ billion (USD),-1
7,Data Scientist,-1,"Hitachi Vantara combines technology, intellect...",2.7,Hitachi Vantara Corporation\n2.7,Bengaluru,-1,5001 to 10000 Employees,1989,Subsidiary or Business Segment,IT Services,Information Technology,Unknown / Non-Applicable,-1
8,Data Scientist,-1,Company Description\n\nnull,3.4,WNS Global Services\n3.4,Mumbai,-1,10000+ Employees,1996,Company - Public,Staffing & Outsourcing,Business Services,$100 to $500 million (USD),-1
9,Data Scientist,-1,"Date: Aug 13, 2020\n\nAs the tech firm that cr...",3.9,Telefonaktiebolaget LM Ericsson\n3.9,Noida,-1,10000+ Employees,1876,Company - Public,Telecommunications Services,Telecommunications,$10+ billion (USD),-1
